In [1]:
import geopandas as gpd
import functools as ft
import fiona
import os
import pandas as pd

ERROR 1: PROJ: proj_create_from_database: Open of /opt/conda/share/proj failed


In [2]:
dfs = []

for layer in fiona.listlayers('../arcgis/Bouwgrond.gdb/'):
    df = gpd.read_file('../arcgis/Bouwgrond.gdb/', layer=layer)
    df = df[['gm_naam', 'AREA', 'SUM']].copy()
    df = df.rename(columns={'SUM': 'sum_' + layer,
                            'AREA': 'gm_area',
                            'gm_naam': 'gemeente'
                           })
    dfs.append(df)

df = ft.reduce(lambda left, right: pd.merge(left, right, on=['gemeente', 'gm_area']), dfs)

In [5]:
df.columns = df.columns.str.lower()
df = df.reindex(sorted(df.columns), axis=1)


In [6]:
df

,gemeente,gm_area,sum_zonal_gem_var1,sum_zonal_gem_var2,sum_zonal_gem_var3,sum_zonal_gem_var4,sum_zonal_gem_var5,sum_zonal_gem_var6
0,Sluis,282980000.0,0.0,0.0,0.0,0.0,0.0,0.0
1,Zuidplas,64210000.0,1536.0,738.0,115.0,282.0,237.0,116.0
2,Waterland,55880000.0,676.0,440.0,9.0,98.0,98.0,9.0
3,Wageningen,32360000.0,270.0,149.0,29.0,120.0,108.0,31.0
4,Hengelo,61810000.0,36.0,22.0,7.0,36.0,22.0,7.0
...,...,...,...,...,...,...,...,...
340,Maashorst,138250000.0,145.0,108.0,69.0,146.0,107.0,68.0
341,Bunschoten,30800000.0,33.0,24.0,16.0,122.0,67.0,16.0
342,Sittard-Geleen,80590000.0,72.0,42.0,11.0,72.0,41.0,10.0
343,Vaals,23900000.0,0.0,0.0,0.0,0.0,0.0,0.0


In [31]:
df['dif_3_6_perc'] = df['sum_zonal_gem_var6'] / df['sum_zonal_gem_var3']
df['dif_3_6_abs'] = df['sum_zonal_gem_var3'] - df['sum_zonal_gem_var6']

In [33]:
df[['gemeente', 
    'sum_zonal_gem_var3', 
    'sum_zonal_gem_var6', 
    'dif_3_6_abs']]\
.dropna(subset='dif_3_6_abs')\
.sort_values(by='dif_3_6_abs', ascending=False)

,gemeente,sum_zonal_gem_var3,sum_zonal_gem_var6,dif_3_6_abs
92,Almere,847.0,466.0,381.0
54,Utrecht,421.0,258.0,163.0
296,Lansingerland,341.0,214.0,127.0
204,Nijmegen,165.0,94.0,71.0
15,Zaanstad,62.0,5.0,57.0
...,...,...,...,...
38,Kaag en Braassem,60.0,138.0,-78.0
71,IJsselstein,10.0,100.0,-90.0
188,Hoeksche Waard,12.0,113.0,-101.0
294,Haarlemmermeer,217.0,392.0,-175.0
